In [1]:
import os
import shutil
import json
import ray
from ray.rllib.algorithms.ppo import PPOConfig
from ray.rllib.env.wrappers.pettingzoo_env import ParallelPettingZooEnv
from ray.rllib.env import PettingZooEnv
from ray.tune.registry import register_env
from spoiled_broth.rl.game_env import GameEnv
from ray.rllib.algorithms.callbacks import DefaultCallbacks
from collections import defaultdict
from pathlib import Path
import supersuit as ss
import numpy as np
import csv
import sys

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [ ]:
rl_model = 'PPO'
map_nr = 'kitchen'
n_iterations = 20
save_every_n_iterations = 5
w1_1 = float(1.0)
w1_2 = float(0.0)
w2_1 = float(0.7071)
w2_2 = float(0.7071)

# Define reward weights
reward_weights = {
    "ai_rl_1": (w1_1, w1_2),
    "ai_rl_2": (w2_1, w2_2),
}

In [ ]:
brigit = "/mnt/lustre/home/samuloza"
base_save_dir = f"/data/samuel_lozano/cooked/saved_models/map_{map_nr}/"
Path(base_save_dir).mkdir(parents=True, exist_ok=True)

In [ ]:
def env_creator(config):
    # Your existing GameEnv class goes here
    return GameEnv(**config)

#def env_creator_2(config):
#    env = GameEnv(reward_weights=reward_weights, map_nr=map_nr)
#    env = ss.pad_observations_v0(env)
#    env = ss.pad_action_space_v0(env)
#    return PettingZooEnv(env)

In [5]:
# Initialize Ray
os.environ["RAY_TMPDIR"] = f"/data/samuel_lozano/tmp_ray/"
ray.shutdown()
ray.init(ignore_reinit_error=True, _temp_dir=os.environ["RAY_TMPDIR"]
         )

# Register the environment with RLLib
register_env("spoiled_broth", lambda config: ParallelPettingZooEnv(env_creator(config)))


2025-05-21 16:41:16,336	INFO worker.py:1888 -- Started a local Ray instance.


(MultiAgentEnvRunner pid=2372624) Weighted rewards:
(MultiAgentEnvRunner pid=2372624) 
(MultiAgentEnvRunner pid=2372624) Agent 1: (1.0, 0.0)
(MultiAgentEnvRunner pid=2372624) 
(MultiAgentEnvRunner pid=2372624) Agent 2: (0.7071, 0.7071)
(MultiAgentEnvRunner pid=2372624) 


(MultiAgentEnvRunner pid=2372624) 2025-05-21 16:42:42,364	WARNING deprecation.py:50 -- DeprecationWarning: `RLModule(config=[RLModuleConfig object])` has been deprecated. Use `RLModule(observation_space=.., action_space=.., inference_only=.., model_config=.., catalog_class=..)` instead. This will raise an error in the future!


(MultiAgentEnvRunner pid=2372605) 
(MultiAgentEnvRunner pid=2372605) 
(MultiAgentEnvRunner pid=2372605) 
(MultiAgentEnvRunner pid=2372605) Weighted rewards:
(MultiAgentEnvRunner pid=2372605) Agent 2: (0.7071, 0.7071) [repeated 2x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)


(MultiAgentEnvRunner pid=2372605) 2025-05-21 16:42:59,679	WARNING deprecation.py:50 -- DeprecationWarning: `RLModule(config=[RLModuleConfig object])` has been deprecated. Use `RLModule(observation_space=.., action_space=.., inference_only=.., model_config=.., catalog_class=..)` instead. This will raise an error in the future!
(MultiAgentEnvRunner pid=2372624) 2025-05-21 16:43:04,829	ERROR actor_manager.py:187 -- Worker exception caught during `apply()`: 'MultiAgentEpisode' object has no attribute 'user_data'
(MultiAgentEnvRunner pid=2372624) Traceback (most recent call last):
(MultiAgentEnvRunner pid=2372624)   File "/home/samuel_lozano/anaconda3/envs/cooked/lib/python3.13/site-packages/ray/rllib/utils/actor_manager.py", line 183, in apply
(MultiAgentEnvRunner pid=2372624)     return func(self, *args, **kwargs)
(MultiAgentEnvRunner pid=2372624)   File "/home/samuel_lozano/anaconda3/envs/cooked/lib/python3.13/site-packages/ray/rllib/execution/rollout_ops.py", line 110, in <lambda>
(Mult

(raylet) A worker died or was killed while executing a task by an unexpected system error. To troubleshoot the problem, check the logs for the dead worker. RayTask ID: ffffffffffffffff5c051ad98734854ff53d66f701000000 Worker ID: afb64deae584a4042e7bc904cd47e43e69f13dfd2a55105e99e17869 Node ID: 7345a0219eb83c8d67279cebec9eb09e2417fd520d2b277263d82e9f Worker IP address: 10.117.132.28 Worker port: 42277 Worker PID: 2372605 Worker exit type: SYSTEM_ERROR Worker exit detail: Worker exits unexpectedly. Worker exits with an exit code 1.


In [ ]:
# Verify observation space consistency
def verify_observation_space():
    test_env = env_creator({"reward_weights": reward_weights, "map_nr": map_nr})
    obs_space = test_env.observation_space("ai_rl_1")
    sample_obs = test_env.reset()[0]["ai_rl_1"]
    
    print(f"Observation space: {obs_space}")
    print(f"Sample observation shape: {sample_obs.shape}")
    print(f"Sample observation min/max: {np.min(sample_obs)}, {np.max(sample_obs)}")
    
    assert obs_space.contains(sample_obs), "Observation doesn't match observation space!"
    test_env.close()
    return obs_space

# Get verified observation space
obs_space = verify_observation_space()

Weighted rewards:

Agent 1: (1.0, 0.0)

Agent 2: (0.7071, 0.7071)

Observation space: Box(0.0, 50.0, (1668,), float32)
Sample observation shape: (1668,)
Sample observation min/max: 0.0, 50.0


In [7]:
# Define separate policies for each agent
def policy_mapping_fn(agent_id, episode=None, worker=None, **kwargs):
    return f"policy_{agent_id}"

In [8]:
class PureRewardFromInfoCallback(DefaultCallbacks):
    def on_episode_start(self, *, episode, **kwargs):
        episode.user_data["pure_rewards"] = defaultdict(float)

    def on_postprocess_trajectory(self, *, episode, agent_id, policy_id, postprocessed_batch, original_batches, **kwargs):
        _, infos = original_batches[agent_id]
        for info in infos:
            if "pure_reward" in info:
                episode.user_data["pure_rewards"][agent_id] += info["pure_reward"]

    def on_episode_end(self, *, episode, **kwargs):
        for agent_id, pure_reward in episode.user_data["pure_rewards"].items():
            episode.custom_metrics[f"pure_reward_{agent_id}"] = pure_reward

        episode.custom_metrics["pure_reward_mean"] = sum(episode.user_data["pure_rewards"].values())

In [ ]:
# Configuration for multi-agent training
config = (
    PPOConfig()
    .api_stack(
        enable_rl_module_and_learner=True,
        enable_env_runner_and_connector_v2=True
    )
    .environment(
        env="spoiled_broth",
        env_config={
            "reward_weights": reward_weights,
            "map_nr": map_nr
        },
        clip_actions=True,
    )
    .multi_agent(
        policies={
            "policy_ai_rl_1": (
                None,  # Use default PPO policy
                env_creator({"map_nr": map_nr}).observation_space("ai_rl_1"),
                env_creator({"map_nr": map_nr}).action_space("ai_rl_1"),
                {}
            ),
            "policy_ai_rl_2": (
                None,  # Use default PPO policy
                env_creator({"map_nr": map_nr}).observation_space("ai_rl_2"),
                env_creator({"map_nr": map_nr}).action_space("ai_rl_2"),
                {}
            )
        },
        policy_mapping_fn=policy_mapping_fn,
        policies_to_train=["policy_ai_rl_1", "policy_ai_rl_2"]
    )
    #.resources(num_gpus=1)  # Set to 1 if you have a GPU
    .env_runners(num_env_runners=2)
    .training(
        train_batch_size=4000,
        minibatch_size=500,
        num_epochs=10
    )
    .callbacks(PureRewardFromInfoCallback)
)

Weighted rewards:

Agent 1: (1.0, 0.0)

Agent 2: (1.0, 0.0)

Weighted rewards:

Agent 1: (1.0, 0.0)

Agent 2: (1.0, 0.0)

Weighted rewards:

Agent 1: (1.0, 0.0)

Agent 2: (1.0, 0.0)

Weighted rewards:

Agent 1: (1.0, 0.0)

Agent 2: (1.0, 0.0)



In [10]:
# Build algorithm with error handling
try:
    algo = config.build_algo()
    full_run_dir = algo.logdir
    NAME_RAY = Path(full_run_dir).name
    print(f"Algorithm built successfully: {NAME_RAY}")
except Exception as e:
    print(f"Error building algorithm: {e}")
    raise

2025-05-21 16:41:27,649	WARNING algorithm_config.py:4968 -- You are running PPO on the new API stack! This is the new default behavior for this algorithm. If you don't want to use the new API stack, set `config.api_stack(enable_rl_module_and_learner=False,enable_env_runner_and_connector_v2=False)`. For a detailed migration guide, see here: https://docs.ray.io/en/master/rllib/new-api-stack-migration-guide.html
2025-05-21 16:43:01,504	WARNING deprecation.py:50 -- DeprecationWarning: `RLModule(config=[RLModuleConfig object])` has been deprecated. Use `RLModule(observation_space=.., action_space=.., inference_only=.., model_config=.., catalog_class=..)` instead. This will raise an error in the future!
2025-05-21 16:43:04,420	INFO trainable.py:160 -- Trainable.setup took 96.705 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.


Algorithm built successfully: PPO_spoiled_broth_2025-05-21_16-41-27_v692f15


In [ ]:
# Save training config
training_dir = Path(f"{base_save_dir}{NAME_RAY}")
training_dir.mkdir(parents=True, exist_ok=True)
shutil.copytree(full_run_dir, training_dir, dirs_exist_ok=True)

def safe_config_dict(d):
    def make_serializable(o):
        try:
            json.dumps(o)
            return o
        except TypeError:
            return str(o)

    return {
        k: make_serializable(v)
        for k, v in d.items()
    }

config_path = f"{training_dir}/config.txt"
with open(config_path, "w") as f:
    f.write("==== Training parameters ====\n")
    f.write(f"RL model: {rl_model}\n")
    f.write(f"Map id: {map_nr}\n")
    f.write(f"Number of iterations: {n_iterations}\n")
    f.write(f"Saved every N iterations: {save_every_n_iterations}\n")
    f.write(f"Reward weights ai_rl_1: {w1_1}, {w1_2}\n")
    f.write(f"Reward weights ai_rl_2: {w2_1}, {w2_2}\n")
    f.write("\n==== Complete configuration ====\n")
    f.write(json.dumps(safe_config_dict(config.to_dict()), indent=4))

csv_file_path = f'{training_dir}/reward_data.csv'
with open(csv_file_path, mode='a', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["iteration", "total_reward", "reward_agent_1", "reward_agent_2", "pure_reward_mean", "pure_reward_agent_1", "pure_reward_agent_2"])

In [12]:
for i in range(n_iterations): 
    result = algo.train()
    reward_agent_1 = result['env_runners']['module_episode_returns_mean'].get('policy_ai_rl_1', 0)
    reward_agent_2 = result['env_runners']['module_episode_returns_mean'].get('policy_ai_rl_2', 0)
    total_reward = result['env_runners']['episode_return_mean']
    custom_metrics = result.get("custom_metrics", {})
    pure_reward_agent_1 = custom_metrics.get("pure_reward_agent_1", 0.0)
    pure_reward_agent_2 = custom_metrics.get("pure_reward_agent_2", 0.0)
    pure_reward_mean = custom_metrics.get("pure_reward_mean", 0.0)

    print(f"Iteration {i}:")
    print(f"  Agent 1 reward: {reward_agent_1}")
    print(f"  Agent 2 reward: {reward_agent_2}")
    print(f"  Total reward: {total_reward}")
    print(f"  Pure reward agent 1: {pure_reward_agent_1}")
    print(f"  Pure reward agent 2: {pure_reward_agent_2}")
    print(f"  Total pure reward: {pure_reward_mean}")

    with open(csv_file_path, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([i, total_reward, reward_agent_1, reward_agent_2, pure_reward_mean, pure_reward_agent_1, pure_reward_agent_2])

    # Save checkpoint
    if i % save_every_n_iterations == 0:
        checkpoint_obtained = algo.save()
        checkpoint_path = Path(checkpoint_obtained.checkpoint.path)
        
        custom_checkpoint_dir = Path(f"{training_dir}/Checkpoint_{i}")
        custom_checkpoint_dir.mkdir(parents=True, exist_ok=True)

        for item in checkpoint_path.iterdir():
            dst = custom_checkpoint_dir / item.name
            if item.is_dir():
                shutil.copytree(item, dst, dirs_exist_ok=True)
            else:
                shutil.copy2(item, dst)

        print(f"Checkpoint {i} saved in {custom_checkpoint_dir}")

2025-05-21 16:44:04,620	WARNING rollout_ops.py:121 -- No samples returned from remote workers. If you have a slow environment or model, consider increasing the `sample_timeout_s` or decreasing the `rollout_fragment_length` in `AlgorithmConfig.env_runners().
2025-05-21 16:44:07,570	ERROR actor_manager.py:873 -- Ray error (The actor eeddc41a20a265ebc9effc9e01000000 is unavailable: The actor is temporarily unavailable: IOError: Fail all inflight tasks due to actor state change.. The task may or maynot have been executed on the actor.), taking actor 1 out of service.
2025-05-21 16:44:07,571	ERROR actor_manager.py:873 -- Ray error (The actor 5c051ad98734854ff53d66f701000000 is unavailable: The actor is temporarily unavailable: IOError: Fail all inflight tasks due to actor state change.. The task may or maynot have been executed on the actor.), taking actor 2 out of service.
2025-05-21 16:44:07,572	ERROR actor_manager.py:674 -- The actor eeddc41a20a265ebc9effc9e01000000 is unavailable: The a

KeyError: 'env_runners'

# Reload params

In [29]:
from ray.rllib.algorithms.algorithm import Algorithm
import torch

In [ ]:
reload_map_nr = 1

training_id = "PPO_spoiled_broth_2025-05-18_22-22-47j6l54y_m"

checkpoint_nr = 0

In [ ]:
# Restaura el algoritmo desde un checkpoint
reload_checkpoint_dir = f"/data/samuel_lozano/cooked/saved_models/map_{reload_map_nr}/{training_id}/Checkpoint_{checkpoint_nr}/"

reload_algo = Algorithm.from_checkpoint(reload_checkpoint_dir)

policy_module_1 = reload_algo.get_policy("policy_ai_rl_1")
policy_module_2 = reload_algo.get_policy("policy_ai_rl_2")

# Comparar pesos (si estás usando PyTorch)
params1 = policy_module_1.get_parameters()
params2 = policy_module_2.get_parameters()

2025-05-18 22:48:33,277	INFO trainable.py:160 -- Trainable.setup took 14.885 seconds. If your trainable is slow to initialize, consider setting reuse_actors=True to reduce actor creation overheads.


AttributeError: 'MultiAgentEnvRunner' object has no attribute 'get_policy'

In [ ]:
params1 = dict(policy_module_1.model.named_parameters())
params2 = dict(policy_module_2.model.named_parameters())

for name in params1:
    if name in params2:
        are_equal = torch.allclose(params1[name], params2[name])
        print(f"{name}: {'Equal' if are_equal else 'Different'}")

<bound method Deprecated.<locals>._inner.<locals>._ctor of PPO(env=spoiled_broth; env-runners=2; learners=0; multi-agent=True)>


In [ ]:
def params_to_vector(params):
    # Convierte diccionario de parámetros en un vector concatenado (PyTorch tensors)
    vectors = []
    for key, tensor in params.items():
        if isinstance(tensor, torch.Tensor):
            vectors.append(tensor.flatten())
        else:
            # Si no es tensor, intenta convertirlo o ignora
            pass
    return torch.cat(vectors)

v1 = params_to_vector(params1)
v2 = params_to_vector(params2)

diff = torch.norm(v1 - v2).item()
print(f"Diferencia entre parámetros de ambas políticas: {diff}")